In [1]:
import random
import numpy as np

In [121]:
class TheGame:

    def __init__(self):
        self.drawing_pile = [i for i in range(2,100)]
        random.shuffle(self.drawing_pile)

        self.hand = self.drawing_pile[:8]
        self.hand.sort()
        self.drawing_pile = self.drawing_pile[8:]

        self.stacks = {"up1": [1],
                       "up2": [1],
                       "down1": [100],
                       "down2": [100]}

    def recognizing_ten_diffs(self, number):
        for nr in self.hand:
            if abs(nr - number) == 10: ###### <- niet af nog
                diff_item = [number, nr]
                diff_item.sort()
                return diff_item, nr
        return None
    
    def ten_moves(self):
        ten_diffs_lst = []
        for nr in self.hand:
            ten_diffs = self.recognizing_ten_diffs(nr)

            if ten_diffs != None:
                self.hand.remove(nr)
                self.hand.remove(ten_diffs[1])
                ten_diffs_lst.append(ten_diffs[0])

        self.hand.extend(ten_diffs_lst)
                
        return self.hand
    
    def draw_from_pile(self):
        if len(self.drawing_pile) > 0:
            self.hand.append(self.drawing_pile[0])
            self.drawing_pile = self.drawing_pile[1:]
            self.hand.sort()
            self.hand = self.hand.ten_moves()
    
    def optimal_move(matrix):
        return list(zip(*np.where(matrix == matrix.min())))
    
    def analyze_move(self):
        playing_matrix = np.matrix(np.ones((4, len(self.hand))) * np.inf)

        elem_counter = 0
        for elem in self.hand:
            stack_counter = 0
            if type(elem) == int:
                for key, val in self.stacks.items():
                    if key.startswith("up"):
                        if elem > val[0]:
                            playing_matrix[stack_counter, elem_counter] = elem - val[0]

                    else:
                        if elem < val[0]:
                            playing_matrix[stack_counter, elem_counter] = val[0] - elem

                    stack_counter += 1

            else:
                for key, val in self.stacks.items():
                    if key.startswith("up"):
                        if max(elem) > val[0]:
                            playing_matrix[stack_counter, elem_counter] = min(elem) - val[0]

                    else:
                        if elem < val[0]:
                            playing_matrix[stack_counter, elem_counter] = val[0] - max(elem)

                    stack_counter += 1

            elem_counter += 1

        return playing_matrix
    
    def optimal_move(self, matrix):
        return list(zip(*np.where(matrix == matrix.min())))
    
    def play_move(self):
        matrix = self.analyze_move()
        best_move = self.optimal_move(matrix)
        stacklist = list(game_instance.stacks.keys())

        if best_move == np.inf:
            print(rf"No move possible anymore. Game ended with a total of {len(self.hand) + len(self.drawing_pile)} left.")

        elif len(best_move) != 1:
            tied_best_cards = []
            stacks_for_tied_best_cards = []
            dist_from_base = []

            for move in best_move:
                stack, card = list(move)[0], list(move)[1]
                tied_best_cards.append(self.hand[card])
                stacks_for_tied_best_cards.append(stack)

                if stack in [0,1]:
                    dist_from_base.append(card-1)
                else:
                    dist_from_base.append(100-card)

                choice = dist_from_base.index(min(dist_from_base))
            
            best_move = best_move[choice]

        stack, card = list(best_move)[0], list(best_move)[1]

        if type(self.hand[card]) == int:
            self.stacks[stacklist[stack]].insert(0, self.hand[card])
            self.hand.remove(self.hand[card])
            return 1, rf"New stacks:{self.stacks}, New hand: {self.hand}"

        else:
            if stack in [0,1]:
                for item in reversed(self.hand[card]):
                    self.stacks[stacklist[stack]].insert(0, item)
                self.hand.remove(self.hand[card])
            else:
                for item in self.hand[card]:
                    self.stacks[stacklist[stack]].insert(0, item)
            self.hand.remove(self.hand[card])
            return 2, rf"New stacks:{self.stacks}, New hand: {self.hand}"                    

game_instance = TheGame()
print(game_instance.hand)
game_instance.play_move()

[13, 30, 36, 45, 51, 55, 63, 64]


(1,
 "New stacks:{'up1': [13, 1], 'up2': [1], 'down1': [100], 'down2': [100]}, New hand: [30, 36, 45, 51, 55, 63, 64]")

In [71]:
list(game_instance.stacks.keys())

['up1', 'up2', 'down1', 'down2']